In [1]:
#####################################################################################
# reddit_api_praw_bulk_script.py
#
# Description:
# Given an input CSV file of search terms and subreddits to include in the search,
# this script scrapes comments on Reddit submissions / threads.
#
# Homepage URL: https://www.reddit.com/
# Tools: Reddit API Praw
# Data Organizer: Pandas
#
# Version: 2.2
# Date Created: 08/22/2019
# Last Modified: 09/09/2019
#####################################################################################

In [2]:
##### IMPORTING PACKAGES ############################################################

import datetime as dt
import pandas as pd
import re

import praw
from praw.models import MoreComments

#####################################################################################

In [7]:
##### GLOBAL VARIABLES ##############################################################

# Path to the project folder (CHANGE AS NEEDED)
# Type: string ending with "/"
PROJECT_FOLDER = "C://Users/kathleen.trinh/Documents/Nissan/LEAF/Reddit/"

# Folder name of folder holding search result files (CHANGE AS NEEDED)
# Type: string ending with "/"
SUBREDDIT_FOLDER = "Subreddits/"

# Folder name of folder holding submission comments (CHANGE AS NEEDED)
# Type: string ending with "/"
SUBMISSIONS_FOLDER = "Submissions/"

# File name of the .csv input file (CHANGE AS NEEDED)
# Type: string
# Note: Please end it with ".csv"
IN_FILENAME = "LEAF Social Listening Sources - Reddit.csv"

# Sample size of the input dataframe to use for script execution
# Type: integer
SAMPLE_SIZE = 0

# Date of script execution (YYMMDD) used for output filename purposes (CHANGE AS NEEDED)
# Type: string
DATE = "210914"

#####################################################################################

In [4]:
##### FUNCTIONS #####################################################################

def get_subreddit_names(input_df, rem_rslash=True):
    subRs = [col for col in input_df][1:]
    if rem_rslash == True:
        subRs = [sub.split("r/")[-1] for sub in subRs]
    return subRs

def get_search_terms(input_df, as_dict=False):
    search_terms = list(input_df.loc[:, "Search Term"])
    #search_terms.remove("ENTIRE SUBREDDIT")
    search_terms = [term.split("\n")[0] for term in search_terms]
    if as_dict == False:
        return search_terms
    else:
        index_vals = [i for i in range(0, len(search_terms))]
        search_terms_dict = dict(zip(search_terms, index_vals))
        return search_terms_dict
    
def get_subR_search_dict(input_df, subR_name, search_terms):
    search_values = input_df.loc[:, subR_name]
    search_values = [str(x).upper() for x in search_values]
    subR_search_dict = dict(zip(search_terms, search_values))
    return subR_search_dict

#####################################################################################

def get_date(created):
    return dt.datetime.fromtimestamp(created)

def check_dict_len(length, mydict):
    for k in mydict:
        if len(mydict[k]) != length:
            return False
    return True

def print_submission_info(reddit_submission):
    print(reddit_submission.id)
    print(reddit_submission.title)
    #print(reddit_submission.body)
    print(reddit_submission.author)
    print(reddit_submission.num_comments)
    print(reddit_submission.score)
    print(reddit_submission.upvote_ratio)
    print(reddit_submission.distinguished)
    print(reddit_submission.stickied)
    print(reddit_submission.edited)
    print(reddit_submission.url)
    print(reddit_submission.created)
    
def make_subreddit_filename(subreddit_name, search_query):
    if subreddit_name == "":
        subreddit_name = "all"
    if search_query == "ENTIRE SUBREDDIT":
        search_query = "all"
    filename = "Reddit Subreddit_" + subreddit_name + " Search_" + search_query + " " + DATE
    return filename

def make_submission_filename(subreddit_name, search_query, submission_ID):
    if subreddit_name == "":
        subreddit_name = "ALL"
    if search_query == "ENTIRE SUBREDDIT":
        search_query = "ALL"
    filename = "Reddit Subreddit_" + subreddit_name + " Search_" + search_query + " SubID_" + submission_ID + " " + DATE
    return filename

def make_submission_filename(submission_ID):
    filename = "Reddit SubID_" + submission_ID + " " + DATE
    return filename
    
#####################################################################################

# Get threads in subreddit by either .top or .search
# NOTES:
# 1. Default value for limit is limit=100
#    limit – Can be any number in 1-1000
#    Max value for limit is limit=1000 -- "Reddit’s request limit* is 1000"
# 2. Default value for time_filter is time_filter='all'
#    time_filter – Can be one of: all, day, hour, month, week, year (default: all)
# 3. subreddit.search(search_query) searches for threads containing "search_query" in "subreddit"
#    When using the function ".search()", results do not have submission.body,
#    so please comment out lines of code that include "body".
#    Using search seems to return 100 results.
# EXAMPLES: top_subreddit = subreddit.top()
#           top_500_subreddit = subreddit.top(limit=500)
#           search_subreddit_results = subreddit.search(search_query, limit=None)

def get_submission_IDs(subR_submission_IDs, subreddit, subreddit_name, search_query, make_output=True):
    
    if search_query == "ENTIRE SUBREDDIT":
        my_subreddit = subreddit.top(limit=None)
        search_query = "all"
    else:
        my_subreddit = subreddit.search(search_query, limit=None)
        
    # Set up dictionary to put subreddit info into (if make_output == True)
    topics_dict = {"search_subreddit": [],
                   "search_query": [],
                   "subreddit": [],
                   "id": [],
                   "title": [],
                   "body": [],
                   "author": [],
                   "num_comments": [],
                   "score": [],
                   "upvote_ratio": [],
                   "distinguished": [],
                   "stickied": [],
                   "edited": [],
                   "permalink": [],
                   "url":[],
                   "created": []}
    
    # For each submission in the search result, get the submission ID,
    # and put subreddit info into the dictionary if make_output == True
    counter = 1
    for submission in my_subreddit:
        if submission.num_comments > 0:
            print(str(counter) + ". Getting submission ID for submission #" + str(counter) + " | submission ID = " + submission.id)
            subR_submission_IDs.add(submission.id)
            if make_output == True:
                print("\t Noting info about submission #" + str(counter) + "...")
                topics_dict["search_subreddit"].append(subreddit_name)
                topics_dict["search_query"].append(search_query)
                topics_dict["subreddit"].append(submission.permalink.split("/")[2])
                topics_dict["id"].append(submission.id)
                topics_dict["title"].append(submission.title)
                topics_dict["body"].append(submission.selftext)
                topics_dict["author"].append(submission.author)
                topics_dict["num_comments"].append(submission.num_comments)
                topics_dict["score"].append(submission.score)
                topics_dict["upvote_ratio"].append(submission.upvote_ratio)
                topics_dict["distinguished"].append(submission.distinguished)
                topics_dict["stickied"].append(submission.stickied)
                topics_dict["edited"].append(submission.edited)
                topics_dict["permalink"].append(submission.permalink)
                topics_dict["url"].append(submission.url)
                topics_dict["created"].append(submission.created)
        else:
            print(str(counter) + ". NOT getting submission ID for submission #" + str(counter) + " because it has 0 comments | submission ID = " + submission.id)
        counter += 1
    
    # If make_output == True, make .csv output for the search results in the subreddit
    if make_output == True:
        # Create Pandas DataFrame
        topics_df = pd.DataFrame(topics_dict)
        # Add timestamp column to DataFrame
        _timestamp = topics_df["created"].apply(get_date)
        topics_df = topics_df.assign(timestamp = _timestamp)
        # Export Pandas DataFrame object to a .csv file
        filename = make_subreddit_filename(subreddit_name, search_query)
        print("\nMaking .csv output for info about " + str(len(topics_dict)) + " submissions found in subreddit '" + subreddit_name + "' with search query '" + search_query + "' -- " + filename + ".csv")
        topics_df.to_csv((PROJECT_FOLDER + SUBREDDIT_FOLDER + filename + ".csv"), index=None, header=True, encoding='utf-8-sig')
        
    return subR_submission_IDs

#####################################################################################

def get_comments(reddit, submission_ID):
    
    # Create submission instance
    submission = reddit.submission(id=submission_ID)
    
    # Only need to call this once per submission because:
    # "Calling replace_more() is destructive.
    # Calling it again on the same submission instance has no effect."
    try:
        submission.comments.replace_more(limit=None)
    except:
        print("\tWARNING: Encountered error while trying to replace 'MoreComments'")
        print("\tProceeding with data extraction process...")
    
    # Print number of comments in submission
    print("\tDetected " + str(submission.num_comments) + " comments in submission: " + submission_ID)
    
    # Set up dictionary to put comment info into
    comments_dict = {"my_comment_id": [],
                     "subreddit": [],
                     "link_id": [],
                     "title": [],
                     "id": [],
                     "author": [],
                     "body": [],
                     "is_submitter": [],
                     "score": [],
                     "parent_id": [],
                     "distinguished": [],
                     "stickied": [],
                     "edited": [],
                     "permalink": [],
                     "created": []}
    
    # Get the info for the original post of the thread, which is contained in the submission object
    comments_dict["my_comment_id"].append("Reddit_" + submission.id + "_" + submission.id + "_Com0")
    comments_dict["subreddit"].append(submission.permalink.split("/")[2])
    comments_dict["link_id"].append(submission.id)
    comments_dict["title"].append(submission.title)
    comments_dict["id"].append(submission.id)
    # If the body of the post is empty, then the "body" field is identical to the submission's title
    # (submission.selftext) returns an empty string if the submission is a link post
    if submission.selftext == "":
        comments_dict["body"].append(submission.title)
    # Else, if the body is not empty, the "body" field is constructed as:
    # [submission's title] + [2 newline characters "\n\n"] + [submission's body]
    else:
        comments_dict["body"].append(submission.title + "\n\n" + submission.selftext)
    comments_dict["author"].append(submission.author)
    comments_dict["is_submitter"].append("TRUE")
    comments_dict["score"].append(submission.score)
    comments_dict["parent_id"].append(submission.id)
    comments_dict["distinguished"].append(submission.distinguished)
    comments_dict["stickied"].append(submission.stickied)
    comments_dict["edited"].append(submission.edited)
    comments_dict["permalink"].append(submission.permalink)
    comments_dict["created"].append(submission.created)
    
    # For each comment in the submission and put comment info into the dictionary
    counter = 1
    for comment in submission.comments.list():
        #print(". ", end = "")
        # Only get the comment's info if the comment has not been deleted or removed
        if ("deleted" not in comment.body) and ("removed" not in comment.body):
            MyCommentID = "Reddit_" + comment.link_id + "_" + comment.id + "_Com" + str(counter)
            comments_dict["my_comment_id"].append(MyCommentID)
            comments_dict["subreddit"].append(comment.permalink.split("/")[2])
            comments_dict["link_id"].append(comment.link_id)
            comments_dict["title"].append(submission.title)
            comments_dict["id"].append(comment.id)
            comments_dict["body"].append(comment.body)
            comments_dict["author"].append(comment.author)
            comments_dict["is_submitter"].append(comment.is_submitter)
            comments_dict["score"].append(comment.score)
            comments_dict["parent_id"].append(comment.parent_id)
            comments_dict["distinguished"].append(comment.distinguished)
            comments_dict["stickied"].append(comment.stickied)
            comments_dict["edited"].append(comment.edited)
            comments_dict["permalink"].append(comment.permalink)
            comments_dict["created"].append(comment.created)
            counter += 1
    #print()
    
    # Create Pandas DataFrame
    comments_df = pd.DataFrame(comments_dict)
    # Add timestamp column to DataFrame
    _timestamp = comments_df["created"].apply(get_date)
    comments_df = comments_df.assign(timestamp = _timestamp)
    # Export Pandas DataFrame object to a .csv file
    filename = make_submission_filename(submission_ID)
    print("\tMaking comments output for submission: " + submission_ID + " -- " + filename + ".csv")
    comments_df.to_csv((PROJECT_FOLDER + SUBMISSIONS_FOLDER + filename + ".csv"), index=None, header=True, encoding='utf-8-sig')
    print("\n" + (50 * "-") + "\n")

#####################################################################################

def search_subreddit(reddit, input_df, subreddit_name, search_terms):
    
    # Make dict for subreddit search info
    subR_search_dict = get_subR_search_dict(input_df, subreddit_name, search_terms)
    
    # Change subreddit_name to empty string if searching "ALL of Reddit"
    if subreddit_name == "ALL of Reddit":
        subreddit_name = "all"
    
    # Create subreddit instance
    subreddit = reddit.subreddit(subreddit_name)
    
    # Set of submission IDs in subreddit to avoid repetition
    subR_submission_IDs = set()
    
    # For each search term, get submission IDs of threads in search result
    for term in search_terms:
        if subR_search_dict[term] == "TRUE":
            print("SEARCH: Searching subreddit '"+ subreddit_name + "' with search query '" + term + "'...\n")
            subR_submission_IDs = get_submission_IDs(subR_submission_IDs, subreddit, subreddit_name, term)
        else:
            print("SEARCH: NOT searching subreddit '"+ subreddit_name + "' with search query '" + term + "' because subR_search_dict[" + term + "] = " + subR_search_dict[term])
        print("\n" + (50 * "-") + "\n")
        
    #print("\n" + (50 * "-") + "\n")
    print("Found a total of", len(subR_submission_IDs), "threads after searches in subreddit:", subreddit_name)
    print("\n" + (50 * "-") + "\n")
        
    # For each unique submission ID found after all searches in the subreddit, extract the comments in the thread
    counter = 1
    for ID in subR_submission_IDs:
        print(str(counter) + ". Getting comment data for submission #" + str(counter) + " | submission ID = " + ID + " ...")
        get_comments(reddit, ID)
        counter += 1

#####################################################################################

In [5]:
##### MAIN FUNCTION #################################################################

def main():
    
    # Load .csv input file
    input_df = pd.read_csv((PROJECT_FOLDER + IN_FILENAME), header=0, encoding='utf-8-sig')
    
    # Get sample size (if applicable)
    if SAMPLE_SIZE > 0:
        input_df = input_df.sample(SAMPLE_SIZE)
        
    # Print input_df
    print("INPUT DATAFRAME:")
    print(input_df)
    print("\n" + (50 * "-") + "\n")
    
    # Get subreddit names from dataframe
    subreddit_names = get_subreddit_names(input_df)
    
    # Get search terms from dataframe
    search_terms = get_search_terms(input_df)
    
    # Create reddit instance
    reddit = praw.Reddit(client_id = "[YOUR_CLIENT_ID]",
                         client_secret = "[YOUR_CLIENT_SECRET]",
                         user_agent = "[YOUR_USER_AGENT]",
                         username = "[YOUR_USERNAME]",
                         password = "[YOUR_PASSWORD]")
    
    # Set of submission IDs to avoid repetition
    #ALL_submission_IDs = set()
    
    # For each subreddit,
    # search the subreddit for the intended search terms,
    # and extract comments in each submission / thread from the overall search results,
    # making an output .csv file for each relevant submission's / thread's comments.
    for subR_name in subreddit_names:
        search_subreddit(reddit, input_df, subR_name, search_terms)
        
    print("Finished Reddit data extraction!")

#####################################################################################

In [8]:
main()

INPUT DATAFRAME:
        Search Term  ALL of Reddit  askcarsales  Autos   cars  Cartalk  \
0  ENTIRE SUBREDDIT          False        False  False  False    False   
1              leaf          False         True  False  False    False   
2       nissan leaf           True        False   True   True     True   

   electriccars  electricvehicles   leaf  MechanicAdvice  Nissan  nissanleaf  \
0         False             False   True           False   False        True   
1          True              True  False           False    True       False   
2         False             False  False            True   False       False   

   UsedCars  whatcarshouldIbuy  
0     False              False  
1      True               True  
2     False              False  

--------------------------------------------------

SEARCH: NOT searching subreddit 'all' with search query 'ENTIRE SUBREDDIT' because subR_search_dict[ENTIRE SUBREDDIT] = FALSE

--------------------------------------------------

S

101. Getting submission ID for submission #101 | submission ID = oz9vwj
	 Noting info about submission #101...
102. Getting submission ID for submission #102 | submission ID = nzkis3
	 Noting info about submission #102...
103. Getting submission ID for submission #103 | submission ID = ouahe6
	 Noting info about submission #103...
104. Getting submission ID for submission #104 | submission ID = ns8khc
	 Noting info about submission #104...
105. Getting submission ID for submission #105 | submission ID = o83ics
	 Noting info about submission #105...
106. Getting submission ID for submission #106 | submission ID = p5eaqr
	 Noting info about submission #106...
107. Getting submission ID for submission #107 | submission ID = okhbom
	 Noting info about submission #107...
108. Getting submission ID for submission #108 | submission ID = osjmpl
	 Noting info about submission #108...
109. Getting submission ID for submission #109 | submission ID = ojs3g3
	 Noting info about submission #109...
1

201. Getting submission ID for submission #201 | submission ID = lvzot0
	 Noting info about submission #201...
202. NOT getting submission ID for submission #202 because it has 0 comments | submission ID = o9ddk6
203. Getting submission ID for submission #203 | submission ID = mj1nl8
	 Noting info about submission #203...
204. Getting submission ID for submission #204 | submission ID = o7dmuq
	 Noting info about submission #204...
205. Getting submission ID for submission #205 | submission ID = nf2co5
	 Noting info about submission #205...
206. Getting submission ID for submission #206 | submission ID = n9uup3
	 Noting info about submission #206...
207. Getting submission ID for submission #207 | submission ID = m7byb5
	 Noting info about submission #207...
208. NOT getting submission ID for submission #208 because it has 0 comments | submission ID = o66num
209. Getting submission ID for submission #209 | submission ID = mjlbj8
	 Noting info about submission #209...
210. NOT getting su

	Detected 3 comments in submission: na8c53
	Making comments output for submission: na8c53 -- Reddit SubID_na8c53 210914.csv

--------------------------------------------------

14. Getting comment data for submission #14 | submission ID = p67pzu ...
	Detected 7 comments in submission: p67pzu
	Making comments output for submission: p67pzu -- Reddit SubID_p67pzu 210914.csv

--------------------------------------------------

15. Getting comment data for submission #15 | submission ID = mb0252 ...
	Detected 8 comments in submission: mb0252
	Making comments output for submission: mb0252 -- Reddit SubID_mb0252 210914.csv

--------------------------------------------------

16. Getting comment data for submission #16 | submission ID = jiaru9 ...
	Detected 203 comments in submission: jiaru9
	Making comments output for submission: jiaru9 -- Reddit SubID_jiaru9 210914.csv

--------------------------------------------------

17. Getting comment data for submission #17 | submission ID = mi1rrp ..

	Detected 32 comments in submission: lms5i6
	Making comments output for submission: lms5i6 -- Reddit SubID_lms5i6 210914.csv

--------------------------------------------------

47. Getting comment data for submission #47 | submission ID = phk4yf ...
	Detected 208 comments in submission: phk4yf
	Making comments output for submission: phk4yf -- Reddit SubID_phk4yf 210914.csv

--------------------------------------------------

48. Getting comment data for submission #48 | submission ID = mde4h7 ...
	Detected 14 comments in submission: mde4h7
	Making comments output for submission: mde4h7 -- Reddit SubID_mde4h7 210914.csv

--------------------------------------------------

49. Getting comment data for submission #49 | submission ID = odm8pk ...
	Detected 57 comments in submission: odm8pk
	Making comments output for submission: odm8pk -- Reddit SubID_odm8pk 210914.csv

--------------------------------------------------

50. Getting comment data for submission #50 | submission ID = nr9pwk

...

	Detected 6 comments in submission: pi49nh
	Making comments output for submission: pi49nh -- Reddit SubID_pi49nh 210914.csv

--------------------------------------------------

86. Getting comment data for submission #86 | submission ID = l221ef ...
	Detected 28 comments in submission: l221ef
	Making comments output for submission: l221ef -- Reddit SubID_l221ef 210914.csv

--------------------------------------------------

87. Getting comment data for submission #87 | submission ID = c8qaa6 ...
	Detected 3 comments in submission: c8qaa6
	Making comments output for submission: c8qaa6 -- Reddit SubID_c8qaa6 210914.csv

--------------------------------------------------

88. Getting comment data for submission #88 | submission ID = 3v3z5e ...
	Detected 6 comments in submission: 3v3z5e
	Making comments output for submission: 3v3z5e -- Reddit SubID_3v3z5e 210914.csv

--------------------------------------------------

89. Getting comment data for submission #89 | submission ID = 2vgeyc ...

	Detected 6 comments in submission: dn79bl
	Making comments output for submission: dn79bl -- Reddit SubID_dn79bl 210914.csv

--------------------------------------------------

119. Getting comment data for submission #119 | submission ID = llzgnm ...
	Detected 6 comments in submission: llzgnm
	Making comments output for submission: llzgnm -- Reddit SubID_llzgnm 210914.csv

--------------------------------------------------

120. Getting comment data for submission #120 | submission ID = pnc32n ...
	Detected 20 comments in submission: pnc32n
	Making comments output for submission: pnc32n -- Reddit SubID_pnc32n 210914.csv

--------------------------------------------------

121. Getting comment data for submission #121 | submission ID = ibsasn ...
	Detected 7 comments in submission: ibsasn
	Making comments output for submission: ibsasn -- Reddit SubID_ibsasn 210914.csv

--------------------------------------------------

122. Getting comment data for submission #122 | submission ID = 5k

	Detected 3 comments in submission: oy8w4i
	Making comments output for submission: oy8w4i -- Reddit SubID_oy8w4i 210914.csv

--------------------------------------------------

152. Getting comment data for submission #152 | submission ID = 8l3mfy ...
	Detected 14 comments in submission: 8l3mfy
	Making comments output for submission: 8l3mfy -- Reddit SubID_8l3mfy 210914.csv

--------------------------------------------------

153. Getting comment data for submission #153 | submission ID = 6llvdt ...
	Detected 11 comments in submission: 6llvdt
	Making comments output for submission: 6llvdt -- Reddit SubID_6llvdt 210914.csv

--------------------------------------------------

154. Getting comment data for submission #154 | submission ID = bmlit6 ...
	Detected 4 comments in submission: bmlit6
	Making comments output for submission: bmlit6 -- Reddit SubID_bmlit6 210914.csv

--------------------------------------------------

155. Getting comment data for submission #155 | submission ID = 9

	Detected 12 comments in submission: i5nsr0
	Making comments output for submission: i5nsr0 -- Reddit SubID_i5nsr0 210914.csv

--------------------------------------------------

185. Getting comment data for submission #185 | submission ID = etmv49 ...
	Detected 10 comments in submission: etmv49
	Making comments output for submission: etmv49 -- Reddit SubID_etmv49 210914.csv

--------------------------------------------------

186. Getting comment data for submission #186 | submission ID = gijos6 ...
	Detected 3 comments in submission: gijos6
	Making comments output for submission: gijos6 -- Reddit SubID_gijos6 210914.csv

--------------------------------------------------

187. Getting comment data for submission #187 | submission ID = lqoapp ...
	Detected 12 comments in submission: lqoapp
	Making comments output for submission: lqoapp -- Reddit SubID_lqoapp 210914.csv

--------------------------------------------------

188. Getting comment data for submission #188 | submission ID = 

	Detected 2 comments in submission: d7hp12
	Making comments output for submission: d7hp12 -- Reddit SubID_d7hp12 210914.csv

--------------------------------------------------

218. Getting comment data for submission #218 | submission ID = p5eaqr ...
	Detected 1 comments in submission: p5eaqr
	Making comments output for submission: p5eaqr -- Reddit SubID_p5eaqr 210914.csv

--------------------------------------------------

219. Getting comment data for submission #219 | submission ID = 6a7okg ...
	Detected 26 comments in submission: 6a7okg
	Making comments output for submission: 6a7okg -- Reddit SubID_6a7okg 210914.csv

--------------------------------------------------

220. Getting comment data for submission #220 | submission ID = faltlz ...
	Detected 5 comments in submission: faltlz
	Making comments output for submission: faltlz -- Reddit SubID_faltlz 210914.csv

--------------------------------------------------

221. Getting comment data for submission #221 | submission ID = d5